In [ ]:
!pip install transformers evaluate accelerate torch huggingface_hub

In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer 
import huggingface_hub

In [ ]:
huggingface_hub.login()

In [ ]:
# model_id = "meta-llama/Llama-3.2-1B-Instruct"
model_id = "Qwen/Qwen2.5-1.5B-Instruct"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)

pipe = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.eos_token_id,
    torch_dtype=torch.bfloat16,
    device="mps",
)

messages = [
    {"role": "system", "content": "너는 사이버대학교의 진학상담 챗봇이다. 항상 한글로 상담사 처럼 대답해"},
    {"role": "user", "content": "너는 누구니?"},
]

outputs = pipe(
    messages,
    max_new_tokens=256,
)

print(outputs[0]["generated_text"][-1]['content'])

In [ ]:
outputs

In [ ]:
chat = outputs[0]["generated_text"]
chat.append({"role": "user", "content": "인공지능 분야 대학원 진학을 하고 싶은데 전망은 어때?"})
outputs = pipe(
    chat,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1]['content'])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)

messages = [
    {"role": "system", "content": "너는 사이버대학교의 진학상담 챗봇이다. 항상 한글로 상담사 처럼 대답해"},
    {"role": "user", "content": "너는 누구니?"},
]
messages_chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
messages_chat

In [ ]:
inputs = tokenizer(messages_chat, return_tensors="pt", add_special_tokens=False)
inputs = {key: tensor.to("mps") for key, tensor in inputs.items()}
model.to("mps")

In [ ]:
inputs

In [ ]:
generation_config = {
            "max_new_tokens": 100,
            "num_beams": 4,
            "no_repeat_ngram_size": 2,
            "early_stopping": True,
            "do_sample": True,
            "temperature": 0.7,
            "pad_token_id": tokenizer.eos_token_id
        }

outputs = model.generate(**inputs,  **generation_config)
decoded_output = tokenizer.decode(outputs[0][inputs['input_ids'].size(1):], skip_special_tokens=True)
print("Decoded output:\n", decoded_output)

In [ ]:
outputs[0]

In [ ]:
inputs['input_ids'].size(1)

In [ ]:
outputs[0][inputs['input_ids'].size(1):].size()